In [1]:
#import librairies

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

from feature_engine.selection import DropDuplicateFeatures, DropCorrelatedFeatures, DropConstantFeatures 

# Feature Shuffling

# Feature Recursive Addition

# Feature Recursive Elimination